In [1]:
require 'nn'
require 'rnn'
require 'image'
require 'optim'
require 'parallel'
require 'cutorch'
require 'cunn'
require 'cunnx'
dl = require 'dataload'

In [2]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

function Tokenize(inputdic)
    --- This function tokenizes the words into a unigram dictionary
    local out = {}
    for k, v in pairs(inputdic) do
        if v ~= 0 then 
            if out[v] == nil then
                out[v] = 1
            else 
                out[v] = 1 + out[v]
            end
        end
    end
    return out
end

function rougeScores(genSummary, refSummary)
    local genTotal = 0
    local refTotal = 0
    local intersection = 0
    -- Inserting the missing keys
    for k, genCount in pairs(genSummary) do
        if refSummary[k] == nil then
            refSummary[k] = 0
        end
    end
    for k, refCount in pairs(refSummary) do
        local genCount = genSummary[k]
        if genCount == nil then 
            genCount = 0 
        end
        intersection = intersection + math.min(refCount, genCount)
        refTotal = refTotal + refCount
        genTotal = genTotal + genCount
    end

    recall = intersection / refTotal
    prec = intersection / genTotal
    if refTotal == 0 then
        recall = 0
    end 
    if genTotal == 0 then
        prec = 0
    end
    -- tmp = {intersection, refTotal, genTotal}
    if recall > 0 or prec > 0 then
        f1 = (2 * recall * prec) / (recall + prec)
    else 
        f1 = 0
    end
    return recall, prec, f1
end

function buildPredsummary(chosenactions, inputsentences, select_index)
    local summary = torch.zeros(inputsentences:size())
    for i=1, chosenactions:size(1) do
        -- the 2 is for the SELECT index, will have to make this more general later
        if chosenactions[i][select_index] == 1 then
            summary[i]:copy(inputsentences[i])
        end
    end    
    return summary
end

function buildPredsummaryFast(chosenactions, inputsentences, select_index)
    local n = inputsentences:size(1)
    local k = inputsentences:size(2)
    local summary = torch.zeros(inputsentences:size())
    actionmatrix = chosenactions:select(2, select_index):clone():resize(n, 1):view(n, 1):expand(n, k):clone()
    --     This line didn't work for whatever reason...gives weird indexing...
    --     actionmatrix = chosenactions:select(2, select_index):resize(1, n):view(n, 1):expand(n, k):clone()
    return actionmatrix:cmul(inputsentences:double())
end

function buildTotalSummary(predsummary, totalPredsummary)
    nps = predsummary:size(1)
    n_l = totalPredsummary:size(2)
    indices = torch.linspace(1, n_l, n_l):long() 
    for i=1, predsummary:size(1) do
        if predsummary[i]:sum() > 0 then 
            minindex = 1
            for j = 1, totalPredsummary[i]:size(1) do 
                if totalPredsummary[i][j] > 0 then
                    minindex = minindex + 1
                end
            end
            lenx = predsummary[i]:size(1)
            totalPredsummary[i][{{minindex, minindex + lenx - 1}}]:copy(predsummary[i])

        end
    end
end

function buildTotalSummaryFast(predsummary, inputTotalSummary, usecuda)
    tmpSummary = inputTotalSummary:clone()
    nps = predsummary:size(1)
    n_l = inputTotalSummary:size(2)    
    indices = torch.linspace(1, n_l, n_l):long()
    if usecuda then
        indices = indices:cuda()
    end
    for i=1, predsummary:size(1) do
        if predsummary[i]:sum() > 0 then
            -- Finding the largest index with a zero
            -- maxindex = torch.max(indices[torch.eq(totalPredsummary[i], 0)])
            -- totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])
            -- Finding the smallest index with a zero
            minindex = torch.min(indices[torch.eq(tmpSummary[i], 0)])
            lenx = predsummary[i]:size(1)
            tmpSummary[i][{{minindex, minindex + lenx - 1}}]:copy(predsummary[i])
        end
    end
    return tmpSummary
end

In [10]:
-- Setting parameters
n = 10
n_s = 5
k = 3
q = 4
a = 1
b = 1000
embDim = 50
gamma = 0.0
SKIP = 1
SELECT = 2
epsilon = 1
nepochs = 100
base_explore_rate = 0.1
end_baserate = torch.round(nepochs * 0.8)

lr = 1e-8
fast = true
usecuda = false
adapt = false
cuts = 4
mem_multiplier = 1
batch_size = 25
delta = cuts/nepochs
print_perf = true

In [11]:
-- Simulating the data

if usecuda then
    Tensor = torch.CudaTensor
    LongTensor = torch.CudaLongTensor   
    ByteTensor = torch.CudaByteTensor
    maskLayer = nn.MaskedSelect():cuda()
    print("...running on GPU")
else
    maskLayer = nn.MaskedSelect()
    torch.setnumthreads(8)
    Tensor = torch.Tensor
    LongTensor = torch.LongTensor
    ByteTensor = torch.ByteTensor
    print("...running on CPU")
end

-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end
-- Optimal predicted summary
trueSummary = LongTensor(n, k * n_s):fill(0)

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)

     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)

    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
    best_sentences = buildPredsummaryFast(true_actions[i], sentences[i], SELECT)
    trueSummary = buildTotalSummaryFast(best_sentences, trueSummary, usecuda)
end

qTokens = {}
for i=1, n do
    qTokens[i] = Tokenize(trueSummary[i]:totable())
end

...running on CPU	


# Scoring the rougue metrics on the simulation

In [12]:
print(rougeScores(Tokenize(trueSummary[1]:totable()), Tokenize(trueSummary[1]:totable())))

1	1	1	


# Building the model

In [13]:
model = buildModel('lstm', b, embDim, 'f1', adapt, usecuda)
-- model = buildModel('lstm', b, embDim, 'f1', false, usecuda)
-- model = buildModel('bow', b, embDim, 'f1', false, false)

params, gradParams = model:getParameters()
if adapt then 
    criterion = nn.ParallelCriterion():add(nn.MSECriterion()):add(nn.BCECriterion())
    criterion['weights'] = {1, 0.25}
else 
    criterion = nn.MSECriterion()
end 

if usecuda then
    criterion = criterion:cuda()
    model = model:cuda()
end

Running LSTM model to learn f1	


# Scoring the model on the sentences

In [14]:
totalPredsummary = {}
qValues = {}
qActions = {}
qPreds = {}
rewards = {}
lossfull = {}
rouguef1 = {}

totalPredsummary = LongTensor(n, n_s * k):fill(0)

memfull = false
curr_memsize = 0
memsize = n * n_s * mem_multiplier
queryMemory = Tensor(memsize, q):fill(0)
qActionMemory = Tensor(memsize, 2):fill(0)
predSummaryMemory = Tensor(memsize, n_s * k):fill(0)
sentenceMemory = Tensor(memsize, k):fill(0)
qPredsMemory = Tensor(memsize, 2):fill(0)
qValuesMemory = Tensor(memsize, 1):fill(0)
rewardMemory = Tensor(memsize, 1):fill(0)

if adapt then
    regPreds = {}
    regMemory = Tensor(memsize, 1):fill(0) 
end

--- Initializing thingss
for i = 1, n_s do
    qPreds[i] = Tensor(n, 2):fill(0) 
    qValues[i] = Tensor(n, 1):fill(0)
    qActions[i] = Tensor(n, 2):fill(0)
    rewards[i] = Tensor(n, 1):fill(0)
    if adapt then
        regPreds[i] = Tensor(n, 1):fill(0)
    end        
end 

if usecuda then
    criterion = criterion:cuda()
    model = model:cuda()
end

for epoch=1, nepochs do
    --- Reset things at the start of each epoch
    for i=1, n_s do
        qPreds[i]:fill(0)
        qValues[i]:fill(0)
        qActions[i]:fill(0)
        rewards[i]:fill(0)
        totalPredsummary:fill(0)
        if adapt then
            regMemory[i]:fill(0)
        end        
    end

    for i=1, n_s do
        totalPreds = model:forward({sentences[i], queries, totalPredsummary})
        
        if adapt then 
            qPreds[i]:copy(totalPreds[1])
            regPreds[i]:copy(totalPreds[2])
        else
            qPreds[i]:copy(totalPreds)
        end

        if torch.uniform(0, 1) <= epsilon then
            --- Need to do something here
            xrand = torch.rand(qPreds[i]:size())
            qActions[i]:select(2, SELECT):copy(torch.ge(xrand:select(2, SELECT), xrand:select(2, SKIP)))
            qActions[i]:select(2, SKIP):copy(torch.ge(xrand:select(2, SKIP), xrand:select(2, SELECT)))
            qValues[i]:copy(maskLayer:forward({totalPreds, qActions[i]:byte() }) )
        else 
            qMax, qindx = torch.max(qPreds[i], 2)  -- Pulling the best actions
            -- Here's the fast way to select the optimal action for each query
            qActions[i]:copy(qActions[i]:scatter(2, qindx, torch.ones(qPreds[i]:size())):clone())
            qValues[i]:copy(qMax)

        end 
        predsummary = buildPredsummaryFast(qActions[i], sentences[i], SELECT)
        totalPredsummary = buildTotalSummaryFast(predsummary, totalPredsummary, usecuda)
        
        for j = 1, n do
            recall, prec, f1 = rougeScores( qTokens[j],
                                            Tokenize(totalPredsummary[j]:totable()))
            rewards[i][j]:fill(f1)
        end
        if i > 1 then
            -- Calculating change in rougue f1
            rewards[i]:copy(rewards[i] - rewards[i-1])
        end
        -- here's the row indexing
        start_row = curr_memsize + 1
        if memsize < (start_row + n) then 
            start_row = memsize - n + 1
            end_row = start_row + n - 1
            memfull = true
            curr_memsize = 0
        else 
            end_row = start_row + n - 1
            curr_memsize = end_row
        end            
        -- Update memory sequentially until it's full then restart updating it
        qActionMemory[{{start_row, end_row}}]:copy(qActions[i])
        predSummaryMemory[{{start_row, end_row}}]:copy(totalPredsummary)
        sentenceMemory[{{start_row, end_row}}]:copy(sentences[i])
        qPredsMemory[{{start_row, end_row}}]:copy(qPreds[i])
        qValuesMemory[{{start_row, end_row}}]:copy(qValues[i])
        queryMemory[{{start_row, end_row}}]:copy(queries)
        
        if adapt then
            regMemory[{{start_row, end_row}}]:copy(regPreds[i])
        end        

    end
    for i=1, n_s do
        if i  < n_s then
            rewardMemory[{{n * (i-1) + 1, n * i}}]:copy(rewards[i] + gamma * rewards[i + 1] )
        else
            rewardMemory[{{n * (i-1) + 1, n * i}}]:copy(rewards[i] )
        end
    end
    -- Adding back the delta for the last one
    rouguef1[epoch] = (rewards[n_s] + rewards[ n_s - 1] ):mean()

    if memfull then 
        memrows = memsize
    else 
        memrows = curr_memsize
    end
    if usecuda then 
        dataloader = dl.TensorLoader({
                        queryMemory[{{1, memrows}}]:cuda(), 
                        sentenceMemory[{{1, memrows}}]:cuda(), 
                        predSummaryMemory[{{1, memrows}}]:cuda(),
                        qPredsMemory[{{1, memrows}}]:cuda(), 
                        ByteTensor(memrows, 2):copy(qActionMemory[{{1, memrows}}]), 
                        qValuesMemory[{{1, memrows}}]:cuda()
                        }, 
                    rewardMemory[{{1, memrows}}]:cuda()
                )
        if adapt then            
            table.insert(dataloader['inputs'], regMemory[{{1, memrows}}]:cuda() )
        end
    else 
        dataloader = dl.TensorLoader({
                    queryMemory[{{1, memrows}}], 
                    sentenceMemory[{{1, memrows}}], 
                    predSummaryMemory[{{1, memrows}}], 
                    qPredsMemory[{{1, memrows}}], 
                    ByteTensor(memrows, 2):copy(qActionMemory[{{1, memrows}}]), 
                    qValuesMemory[{{1, memrows}}]
                    }, 
                rewardMemory[{{1, memrows}}]
            )
        if adapt then
            table.insert(dataloader['inputs'], regMemory[{{1, memrows}}] )
        end
    end
    loss = {}
    c = 1
    for k, xin, reward in dataloader:sampleiter(batch_size, memsize) do
        local function feval(params)
            gradParams:zero()
            if adapt then
                local ignore = model:forward({xin[1], xin[2], xin[3]})
                local predQOnActions = maskLayer:forward({xin[4], xin[5]}) 
                ones = torch.ones(xin[6]:size(1)):resize(xin[6]:size(1))
                if usecuda then
                    ones = ones:cuda()
                end
                lossf = criterion:forward({predQOnActions, xin[7]}, {reward, ones})
                local gradOutput = criterion:backward({predQOnActions, xin[6]}, {reward, ones})
                local gradMaskLayer = maskLayer:backward({xin[4], xin[5]}, gradOutput[1])
                model:backward({xin[1], xin[2], xin[3]}, {gradMaskLayer[1], gradOutput[2]})
            else 
                local ignore = model:forward({xin[1], xin[2], xin[3]})
                local predQOnActions = maskLayer:forward({xin[4], xin[5]}) 
                lossf = criterion:forward(predQOnActions, reward)
                local gradOutput = criterion:backward(predQOnActions, reward)
                local gradMaskLayer = maskLayer:backward({xin[4], xin[5]}, gradOutput)
                model:backward({xin[1], xin[2], xin[3]}, gradMaskLayer[1])
            end 
            return lossf, gradParams
        end
        --- optim.rmsprop returns \theta, f(\theta):= loss function
         _, lossv  = optim.rmsprop(feval, params, optimParams)
        loss[c] = lossv[1]
        c = c + 1
    end

    lossfull[epoch] = Tensor(loss):sum() / #lossv
    if print_perf then
        print(
            string.format('epoch = %i; rougue = %.6f; epsilon = %.6f; loss = %.6f' , 
                epoch, rouguef1[epoch], epsilon, lossfull[epoch])
            )
    end

    if (epsilon - delta) <= base_explore_rate then
        epsilon = base_explore_rate
        if epoch > end_baserate then 
            base_explore_rate = 0.
        end
    else 
        epsilon = epsilon - delta
    end
end

epoch = 1; rougue = 0.441905; epsilon = 1.000000; loss = 0.198384	


epoch = 2; rougue = 0.585238; epsilon = 0.960000; loss = 1447.331028	


epoch = 3; rougue = 0.417778; epsilon = 0.920000; loss = 8.074948	


epoch = 4; rougue = 0.388889; epsilon = 0.880000; loss = 17.963194	


epoch = 5; rougue = 0.387698; epsilon = 0.840000; loss = 14.400504	


...

epoch = 96; rougue = 0.000000; epsilon = 0.000000; loss = 345872843108.208130	


epoch = 97; rougue = 0.000000; epsilon = 0.000000; loss = 413046796190.239807	


epoch = 98; rougue = 0.000000; epsilon = 0.000000; loss = 431917391440.981567	


epoch = 99; rougue = 0.000000; epsilon = 0.000000; loss = 425762789210.167480	


epoch = 100; rougue = 0.000000; epsilon = 0.000000; loss = 488463628267.096680	


In [15]:
Plot = require 'itorch.Plot'

loss = torch.Tensor(lossfull)
rougue = torch.Tensor(rouguef1)
indices = torch.linspace(1, loss:size(1), loss:size(1)):long() 
plot = Plot():line(indices, loss, 'red', 'hi'):title('Plot of loss'):draw()
plot = Plot():line(indices, rougue, 'blue', 'hi'):title('Plot of Rougue-F1'):draw()